In [1]:
import pandas as pd
import random
import datetime
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, colors
from openpyxl.worksheet.table import Table, TableStyleInfo

In [2]:
def import_csv(filename):
    applicants = pd.read_csv(filename)
    return applicants

In [3]:
applicants = import_csv('C:/Users/arthu/Desktop/Seating_Test.csv')
saved_applicants = applicants.copy(deep=True)

In [4]:
#Delete Timestamp Column
applicants = applicants.drop(['Timestamp'], axis=1)

In [5]:
#Add column to indicate wheter added to group
#applicants['Selected'] = False

In [6]:
applicants.head()

,Username,Name,Dietary Requirements,Guests,Priority,Group Code
0,x,Arthur 1,NaN,10,21st,21 1
1,x,Arthur 2,NaN,0,NaN,Group1
2,x,Arthur 3,NaN,0,NaN,Group2
3,x,Arthur 4,NaN,0,NaN,Group3
4,x,Arthur 5,NaN,0,NaN,Group4


In [7]:
#set capacity
capacity = 73 #NEED TO CHANGE IF DON'T HAVE ALL TABLES

#create database for those attending
attending = pd.DataFrame(columns = list(applicants))
#attending_split_groups = pd.DataFrame(columns = list(applicants))

In [8]:
def add_to_guestlist(i, applicants, guestlist):
        
        #applicants.loc[[i],['Selected']] = True
        
        print('looking at index ', i)
        guestlist = guestlist.append(applicants.loc[i],ignore_index = True)
        
        #Consider Guests
        for _ in range(int(applicants.loc[i,'Guests'])):
            guestlist.loc[len(guestlist)] =[
                applicants.loc[i,'Username'],
                'Guest of ' + applicants.loc[i,'Name'],
                'UNKNOWN PLEASE INFORM',
                'N/A',
                'N/A',
                applicants.loc[i,'Group Code']
            ]

        return applicants, guestlist
    
def add_to_guestlist_split(i, applicants, guestlist):
        
        #applicants.loc[[i],['Selected']] = True

        groupname = 'Separated from ' + str(applicants.loc[i,'Group Code'])
        applicants.loc[i,'Group Code'] = groupname
        guestlist = guestlist.append(applicants.loc[i],ignore_index = True)
        
        #Consider Guests
        for _ in range(int(applicants.loc[i,'Guests'])):
            guestlist.loc[len(guestlist)] =[
                applicants.loc[i,'Username'],
                'Guest of ' + applicants.loc[i,'Name'],
                'UNKNOWN PLEASE INFORM',
                'N/A',
                'N/A',
                groupname
            ]  

        return applicants, guestlist   

In [9]:
#Add those with Priority
priority = []
for i in range(len(applicants)-1):
    if applicants.loc[[i],['Priority']].notnull().bool() == True:
        priority.append(i)
        applicants, attending = add_to_guestlist(i, applicants, attending)
applicants = applicants.drop(priority)
applicants.index = range(len(applicants.index))
print(len(applicants))

        


looking at index  0
99


In [10]:
applicants.head()

,Username,Name,Dietary Requirements,Guests,Priority,Group Code
0,x,Arthur 2,NaN,0,NaN,Group1
1,x,Arthur 3,NaN,0,NaN,Group2
2,x,Arthur 4,NaN,0,NaN,Group3
3,x,Arthur 5,NaN,0,NaN,Group4
4,NaN,Arthur 6,NaN,0,NaN,Group5


In [11]:
###Add those without priority

fail = 0
failed = False
num_guests = applicants['Guests'].sum()
print(num_guests)
print(len(applicants)+num_guests)


while len(attending) <= capacity:
    print('Number of Applicants: ', (len(applicants)+applicants['Guests'].sum()))
    print('Attending ',len(attending))
    #print('Attending by splitting groups ', len(attending_split_groups))
    

    if len(attending) == capacity:
        print('Successfully Filled Formal!')
        break
    
    if len(applicants)==0:
        print('Formal Filled With Space to spare')
        break
        
    
    if len(attending) == capacity and failed == True:
        print('Filled Formal by breaking up groups')
        break
    
    if fail > 5:
        print('Failed to fill without splitting groups')
        failed = True

    
    idx = random.randint(0,len(applicants)-1)
    print('index = ',idx)
    
    #Check whether already selected

        
    if failed == True:
        groupnum = 1 + applicants.loc[idx, 'Guests']
        #print('Individual with guests', applicants.loc[idx, 'Name'], groupnum)
        if len(attending) + groupnum <= capacity:
            print('Added ', applicants.loc[idx, 'Name'],' with Guests')
            
            applicants, attending = add_to_guestlist_split(idx, applicants, attending) 
            applicants = applicants.drop([idx])
            applicants.index = range(len(applicants.index))        
    #No Group
    elif applicants.loc[[idx],['Group Code']].isnull().bool() == True:
        groupnum = 1 + applicants.loc[idx, 'Guests']
        #print('Individual with guests', applicants.loc[idx, 'Name'], groupnum)
        if len(attending) + groupnum <= capacity:
            #print('Added ', applicants.loc[idx, 'Name'])
            applicants, attending = add_to_guestlist(idx, applicants, attending)
            applicants = applicants.drop([idx])
            applicants.index = range(len(applicants.index))
        else:
            fail += 1
            print('Failed to fit ', applicants.loc[idx,'Name'], ' with group')
    
    #Has group
    else:
        groupname = applicants.loc[idx, 'Group Code']
        print('Groupname = ', groupname)
           
        groupnum = len(applicants[applicants.loc[:,'Group Code'] == groupname])
        print('Groupnum = ', groupnum)
        groupnum += (applicants[applicants.loc[:,'Group Code'] == groupname])['Guests'].sum()
        print('Group Number considering guests = ', groupnum)
            
        group_idxs = list(applicants[applicants.loc[:,'Group Code'] == groupname].index)
        print('Group_idxs = ', group_idxs)
           
        print('Number with group: ', len(attending) + groupnum)
        if (len(attending) + groupnum) <= capacity:
            for i in group_idxs:
                applicants, attending = add_to_guestlist(i, applicants, attending)
            
            applicants = applicants.drop(group_idxs)
            applicants.index = range(len(applicants.index))
        else:
            fail += 1
            print('unable to fit group: ', groupname)

8
107
Number of Applicants:  107
Attending  11
index =  7
looking at index  7
Number of Applicants:  105
Attending  13
index =  26
looking at index  26
Number of Applicants:  104
Attending  14
index =  77
looking at index  77
Number of Applicants:  103
Attending  15
index =  13
looking at index  13
Number of Applicants:  102
Attending  16
index =  82
looking at index  82
Number of Applicants:  101
Attending  17
index =  17
looking at index  17
Number of Applicants:  100
Attending  18
index =  63
looking at index  63
Number of Applicants:  99
Attending  19
index =  44
looking at index  44
Number of Applicants:  98
Attending  20
index =  68
Groupname =  Group4
Groupnum =  6
Group Number considering guests =  6
Group_idxs =  [3, 11, 35, 40, 68, 88]
Number with group:  26
looking at index  3
looking at index  11
looking at index  35
looking at index  40
looking at index  68
looking at index  88
Number of Applicants:  92
Attending  26
index =  37
Groupname =  Group5
Groupnum =  6
Group Numb

In [12]:
print(len(applicants))
print(len(attending))
#print(len(attending_split_groups))

42
73


In [13]:
#Post Process Tables:
    
attending = attending.drop(['Username', 'Guests', 'Priority'], axis=1)

#ttending_split_groups = attending_split_groups.drop(['Username', 'Guests', 'Priority', 'Selected'], axis=1)

#attending


In [14]:
guest_list = attending

In [15]:
guest_list.head()

,Name,Dietary Requirements,Group Code
0,Arthur 1,NaN,21 1
1,Guest of Arthur 1,UNKNOWN PLEASE INFORM,21 1
2,Guest of Arthur 1,UNKNOWN PLEASE INFORM,21 1
3,Guest of Arthur 1,UNKNOWN PLEASE INFORM,21 1
4,Guest of Arthur 1,UNKNOWN PLEASE INFORM,21 1


In [16]:
#Make List of Groups with size > 4
odd_groups = pd.DataFrame(columns = ['Group Name', 'Number'])
even_groups = pd.DataFrame(columns = ['Group Name', 'Number'])
accounted_groups = []
for i in range(len(guest_list)):
    if guest_list.loc[[i],['Group Code']].isnull().bool() == False and guest_list.loc[i, 'Group Code'] not in accounted_groups:
            groupname = guest_list.loc[i, 'Group Code']
            accounted_groups.append(groupname)
            #print('Groupname = ', groupname)
            
            groupnum = len(guest_list[guest_list.loc[:,'Group Code'] == groupname])
            #print('Groupnum = ', groupnum)

            if groupnum > 4:
                if groupnum % 2 != 0:
                    odd_groups.loc[len(odd_groups)] = [groupname, groupnum]
                else:
                    even_groups.loc[len(even_groups)] = [groupname, groupnum]

print(accounted_groups)
for groups in [odd_groups, even_groups]:
    groups = groups.sort_values('Number', ascending=False)
print(odd_groups)
print(even_groups)

['21 1', 'Group4', 'Group5', 'Group3', 'Group2', 'Group1']
  Group Name Number
0       21 1     11
1     Group1      7
  Group Name Number
0     Group4      6
1     Group5      6
2     Group3      6
3     Group2      6


In [17]:
writer = pd.ExcelWriter('C:/Users/arthu/Desktop/Seating_Plan.xlsx')
saved_applicants.to_excel(writer,'Applicants', na_rep = None, index = False)
attending.to_excel(writer, 'Attending', na_rep = None, index = False)
applicants.to_excel(writer, 'Failed', na_rep = None, index = False)
writer.save()

In [18]:
wb = load_workbook('C:/Users/arthu/Desktop/Seating_Plan.xlsx')
wb.active = 1
ws = wb.active
tab = Table(displayName="Guest_List", ref = "A1:C96")
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
tab.tableStyleInfo = style
print(type(ws))
ws.add_table(tab)
for i in range(2,97):
    for col in ['A', 'B', 'C']:
        ws[col+str(i)].alignment = Alignment(horizontal='center', vertical='center')
wb.save(filename = 'C:/Users/arthu/Desktop/Seating_Plan.xlsx')

<class 'openpyxl.worksheet.worksheet.Worksheet'>


## Part 2: Seating arrangements

In [19]:
if len(odd_groups) > 0 or len(even_groups) > 0: ###Only Runs if there are groups of sufficient size

    #Find Date of Next Friday
    today = datetime.date.today()
    friday = today + datetime.timedelta( (4-today.weekday()) % 7 )
    friday = friday.strftime('%d/%m/%Y') #Formats date to UK style

    wb = load_workbook('C:/Users/arthu/Desktop/Seating_Plan.xlsx')
    wb.create_sheet('Seating Plan')
    
    wb.active = 3

    ws = wb.active


    #ws.title = "Range Names" # Sheet Title

    #Title
    ws.merge_cells('A2:K2')

    title_cell = ws['A2']

    title_cell.alignment = Alignment(horizontal='center', vertical='center')
    title_cell.font = Font(bold=True)

    ws['A2'] = "Hall Seating Plan " + str(friday)


    ####Step 1
    ####Make Map of Hall (This is For All Table, cap = 95)

    #Colour in Tables
    #set blue fill
    darkblueFill = PatternFill(fill_type='solid',
                              start_color=colors.DARKBLUE,
                              end_color=colors.DARKBLUE)
    redFill = PatternFill(fill_type = 'solid',
                          start_color=colors.RED,
                          end_color=colors.RED)
    #all round borders
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))

    for cell in range(4,12):
        ws['B'+str(cell)].fill = darkblueFill
        ws['J'+str(cell)].fill = redFill # Needs to be red if don't have table

        ws['A'+str(cell)].border = thin_border
        ws['C'+str(cell)].border = thin_border

        ws['I'+str(cell)].border = thin_border
        ws['K'+str(cell)].border = thin_border
    for cell in range(16,24):
        ws['B'+str(cell)].fill = darkblueFill
        ws['J'+str(cell)].fill = darkblueFill

        ws['A'+str(cell)].border = thin_border
        ws['C'+str(cell)].border = thin_border

        ws['I'+str(cell)].border = thin_border
        ws['K'+str(cell)].border = thin_border
    for cell in range(7,21):
        ws['F'+str(cell)].fill = darkblueFill
        ws['E'+str(cell)].border = thin_border
        ws['G'+str(cell)].border = thin_border

    ws['B12'].border = thin_border
    ws['B24'].border = thin_border
    ws['J24'].border = thin_border

In [20]:
class Table():
    def __init__(self, identity, start, end, odd, empty=True, uneven_fill=False):
        """
        identifier
            str to represent specific table
        start
            str in form A1 giving the top left seat
        end 
            str in form A1 giving the bot right seat
        odd
            str None, has jutty bit on at bottomm
            top = has jutty bit on at top
            bot = has jutty bit on at bottom
        free
            int number of free seats left
        empty
            bool indicates whether occupied
        
        """
        self.identity = identity
        self.start = start
        self.end = end 
        self.odd = odd
        length = 0
        length = 2*(int(self.end[1:]) - int(self.start[1:]) + 1)
        if self.odd != None:
            length += 1
        self.free = length
        self.empty = True
        self.uneven_fill = uneven_fill
        
    def try_to_seat_odd(self, groupname, groupnum, ws):
        """
        Attempt to find spot at odd table for odd group
        if successful return True
        only deals with empty tables
        """
        if self.empty == True:
            if self.odd == 'top':
                start = self.start
                start_col = start[0]
                opp_col = chr(ord(start_col) + 2)
                start_row = int(start[1:])
                ####colour in sticky out bit
                jutting_col = chr(ord(start_col)+1)
                jutting_row = start_row - 1
                jutting_cell = jutting_col + str(jutting_row)
                #print('jutting cell: ',jutting_cell)
                #print(type(jutting_cell))
                ws[jutting_cell] = groupname
                ###mark main body of group
                row_range = groupnum//2
                end_row = start_row + row_range

                for i in range(start_row, end_row):
                    ws[start_col + str(i)] = groupname
                    ws[opp_col + str(i)] = groupname
                
                #update free space
                self.free -= groupnum
                self.empty = False
                self.start = start_col + str(end_row)
                return True
                
            elif self.odd == 'bot':
                start = self.end # Remember end point is on opposite side of table to start
                start_col = start[0]
                opp_col = chr(ord(start_col) - 2)
                start_row = int(start[1:])                
                ####colour in sticky out bit
                jutting_col = chr(ord(start_col)-1)
                jutting_row = start_row + 1
                jutting_cell = jutting_col + str(jutting_row)
                ws[jutting_cell] = groupname
                ###mark main body of group
                row_range = groupnum//2
                end_row = start_row - row_range

                for i in range(start_row, end_row, -1):
                    #print('Hello! ' + start_col + str(i))
                    ws[start_col + str(i)] = groupname
                    ws[opp_col + str(i)] = groupname
                
                #update free space
                self.free -= groupnum
                self.empty = False
                self.end = start_col + str(end_row)
                return True

        return False

    def try_to_seat_even(self, groupname, groupnum, ws):
        """
        Attempt to find spot at even table for even group
        if successful return true
        """
        if self.empty == True:
            start = self.start
            start_col = start[0]
            opp_col = chr(ord(start_col) + 2)
            start_row = int(start[1:])

            ###mark main body of group
            row_range = groupnum//2
            end_row = start_row + row_range

            for i in range(start_row, end_row):
                ws[start_col + str(i)] = groupname
                ws[opp_col + str(i)] = groupname
                
            #update free space
            self.free -= groupnum
            self.empty = False
            self.start = start_col + str(end_row)
            return True
        return False
    
    def try_to_seat_any_uneven(self, groupname, groupnum, ws):
        """
        Seat group at table with most free space
        if odd number, should indicate filled unevenly
        if already filled unevenly should take this into account
        """
        if self.free >= groupnum:
            print('There is enough space!')
            if self.uneven_fill == True:
                if groupnum % 2 != 0:
                    #odd number of guests
                    start = self.start
                    start_col = start[0]
                    opp_col = chr(ord(start_col) + 2)
                    start_row = int(start[1:])
                    #Fill to make an even fill again 
                    ws[start] = groupname
                    #move down a row
                    start_row += 1
                    row_range = groupnum//2
                    end_row = start_row + row_range
                    for i in range(start_row, end_row):
                        ws[start_col + str(i)] = groupname
                        ws[opp_col + str(i)] = groupname

                    #update free space
                    self.free -= groupnum
                    self.empty = False
                    self.start = start_col + str(end_row)

                    self.uneven_fill = False
                    return True

                else:
                    #even number
                    start = self.start
                    start_col = start[0]
                    opp_col = chr(ord(start_col) + 2)
                    start_row = int(start[1:])
                    #Fill to make an even fill again 
                    ws[start] = groupname
                    #move down a row
                    start_row += 1
                    row_range = (groupnum//2) - 1
                    end_row = start_row + row_range
                    for i in range(start_row, end_row):
                        ws[start_col + str(i)] = groupname
                        ws[opp_col + str(i)] = groupname
                    #need to fill unevenly
                    ws[opp_col + str(end_row)] = groupname

                    #update free space
                    self.free -= groupnum
                    self.empty = False
                    self.start = start_col + str(end_row)
                    return True                
        else:
            return False
    
    def try_to_seat_any_even(self,groupname,groupnum,ws):
        if self.free >= groupnum:
            print('There is enough space!')
            if groupnum % 2 == 0: # even number so leaves uneven_fill unaffected
                start = self.start
                start_col = start[0]
                opp_col = chr(ord(start_col) + 2)
                start_row = int(start[1:])

                ###mark main body of group
                row_range = groupnum//2
                end_row = start_row + row_range

                for i in range(start_row, end_row):
                    ws[start_col + str(i)] = groupname
                    ws[opp_col + str(i)] = groupname
             
                #update free space
                self.free -= groupnum
                self.empty = False
                self.start = start_col + str(end_row)
                return True
            

            elif groupnum % 2 != 0:
                start = self.start
                start_col = start[0]
                opp_col = chr(ord(start_col) + 2)
                start_row = int(start[1:])

                ###mark main body of group
                row_range = groupnum//2
                end_row = start_row + row_range

                for i in range(start_row, end_row):
                    ws[start_col + str(i)] = groupname
                    ws[opp_col + str(i)] = groupname

                #need to do jutty bit
                ws[opp_col + str(end_row)] = groupname
                self.uneven_fill = True

                #update free space
                self.free -= groupnum
                self.empty = False
                self.start = start_col + str(end_row)
                return True                
        else:
            return False

In [21]:
def find_emptiest_table(array):
    emptiest_table = None
    max_length = 0
    for table in array:
        if table.free > max_length:
            max_length = table.free
            emptiest_table = table
    return emptiest_table

a=Table('a', 'A4', 'C11', 'bot')
#b=Table('b', 'I4', 'K11', None) #may not have this table
c=Table('c', 'A16', 'C23', 'bot')
d=Table('d', 'I16', 'K23', 'bot')
e=Table('e', 'E7', 'G20', None)
oddtables = [a, c, d]
eventables = [e]
alltables = [a,c,d,e]
#a = A4 - C11 + B12
#b = I4 - K11
#c = A16 - C23 + B24
#d = I16 - K23 + J24
#e = E7 - G20

if len(odd_groups) > 0 or len(even_groups) > 0: ###Only Runs if there are groups of sufficient size
    print(len(odd_groups))
    for i in range(len(odd_groups)):
        print(i)
        groupname = odd_groups.loc[i, 'Group Name']
        #print(groupname)
        #print(type(groupname))
        groupnum = odd_groups.loc[i, 'Number']
        #print(groupnum)
        
        seated = False
        
        for table in random.sample(oddtables, len(oddtables)):
            #print('Trying Table ', table.identity)
            if table.try_to_seat_odd(groupname, groupnum, ws) == True:
                seated = True
                print('Successfully seated %s at table %s' % (groupname, table.identity))
                break
        
        if seated == False:
            freetable = find_emptiest_table(alltables)
            #print("Table %s has the most space" % (freetable.identity))
            #print('groupnum = %d' % groupnum)
            #print(type(groupnum))
            #print('free space = %d' % freetable.free)
            #print(type(freetable.free))
            
            if freetable.uneven_fill == True:
                if freetable.try_to_seat_any_uneven(groupname,groupnum,ws) == True:
                    seated = True
                    print('Successfully seated %s at table %s' % (groupname, freetable.identity))
                    
                    
            if freetable.uneven_fill == False:       
                if freetable.try_to_seat_any_even(groupname, groupnum, ws) == True:
                    seated = True
                    print('Successfully seated %s at table %s' % (groupname, freetable.identity))
                    
            if seated == False:
                print('FAILED TO SEAT GROUP: ' + groupname)                    
                    
    for i in range(len(even_groups)):
        
        groupname = even_groups.loc[i, 'Group Name']
        #print(groupname)
        #print(type(groupname))
        groupnum = even_groups.loc[i, 'Number']
        #print(groupnum)
        seated = False
        
        for table in random.sample(eventables, len(eventables)):
            print('trying table ', table.identity)
            
            if table.try_to_seat_even(groupname, groupnum, ws) == True:
                seated = True
                print('Successfully seated %s at table %s' % (groupname, table.identity))
                break
            
            if seated == False:
                freetable = find_emptiest_table(alltables)
                #print("Table %s has the most space" % (freetable.identity))
                #print('groupnum = %d' % groupnum)
                #print(type(groupnum))
                #print('free space = %d' % freetable.free)
                #print(type(freetable.free))
                if freetable.uneven_fill == True:
                    if freetable.try_to_seat_any_uneven(groupname,groupnum,ws) == True:
                        seated = True
                        print('Successfully seated %s at table %s' % (groupname, freetable.identity))
                        
                if freetable.uneven_fill == False:       
                    if freetable.try_to_seat_any_even(groupname, groupnum, ws) == True:
                        seated = True
                        print('Successfully seated %s at table %s' % (groupname, freetable.identity))
                        
                if seated == False:
                    print('FAILED TO SEAT GROUP: ' + groupname)
                    

2
0
Successfully seated 21 1 at table c
1
Successfully seated Group1 at table d
trying table  e
Successfully seated Group4 at table e
trying table  e
There is enough space!
Successfully seated Group5 at table e
trying table  e
There is enough space!
Successfully seated Group3 at table a
trying table  e
There is enough space!
Successfully seated Group2 at table e


In [22]:
len(odd_groups)

2

In [23]:
if len(odd_groups) > 0 or len(even_groups) > 0: ###Only Runs if there are groups of sufficient size
    wb.save(filename = 'C:/Users/arthu/Desktop/Seating_Plan.xlsx')